In [28]:
from PIL import Image
import pandas as pd
import os, cv2
import numpy as np

In [ ]:
!pip install Ninja

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing.git

In [ ]:
!git clone https://github.com/Healthcare-Robotics/bodies-at-rest.git

In [ ]:
%cd Self-Correction-Human-Parsing

In [5]:
!mkdir input
!mkdir output
!mkdir dataset

In [ ]:
!chmod +x download_real.sh
!bash download_real.sh

In [21]:
object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/S103/prescribed.p')
print(len(object['RGB']))
print(object['RGB'][0].shape)

48
(880, 440, 3)


In [36]:
path = "/content/bodies-at-rest/data_BR/real/"
folders = os.listdir(path)
w, h = 880, 440

for i, folder in enumerate(folders):
  object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/'+ folder +'/prescribed.p')
  for j in range(0, len(object['RGB'])):
    rgb_img = np.zeros((h, w, 3), dtype=np.uint8)
    rgb_img = object['RGB'][j]
    rgb_img = Image.fromarray(rgb_img, 'RGB')
    rgb_img.save(f'/content/drive/MyDrive/Datasets/Self-Correction-Human-Parsing/{i}{j}.jpg')

In [38]:
!python '/content/Self-Correction-Human-Parsing/simple_extractor.py' --dataset 'pascal' --model-restore '/content/drive/MyDrive/models/exp-schp-201908270938-pascal-person-part.pth' --input-dir '/content/drive/MyDrive/Datasets/Self-Correction-Human-Parsing' --output-dir '/content/drive/MyDrive/Datasets/Self-Correction-Human-Parsing-Segmented'

Evaluating total class number 7 with ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']
100% 917/917 [04:15<00:00,  3.59it/s]


In [47]:
path = "/content/drive/MyDrive/Datasets/Self-Correction-Human-Parsing-Segmented/"
segmented_images = os.listdir(path)

len(segmented_images)

917

In [ ]:
%cd /content/Self-Correction-Human-Parsing/dataset/
!mkdir train_imgs
!mkdir val_imgs

In [50]:
for i ,image in enumerate(segmented_images):
  img = cv2.imread(os.path.join(path, image))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray[img_gray > 0] = 255
  
  if i < 800:
    cv2.imwrite(f"train_imgs/{image}", img_gray)
  else:
    cv2.imwrite(f"val_imgs/{image}", img_gray)